# Day 23: Unstable Diffusion

[*Advent of Code 2022 day 23*](https://adventofcode.com/2022/day/23) and [*solution megathread*]()

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2022/23/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2022%2F23%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')


%load_ext nb_mypy
%nb_mypy On

Version 1.0.4


In [2]:
import common


downloaded = common.refresh()
%store downloaded >downloaded

%load_ext pycodestyle_magic
%pycodestyle_on

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [3]:
from IPython.display import HTML

HTML(downloaded['part1'])

## Comments

...

In [16]:
testdata = """....#..
..###.#
#...#.#
.#...##
#.###..
##.#.##
.#..#..""".splitlines()

testdata2 = """.....
..##.
..#..
.....
..##.
.....""".splitlines()

inputdata = downloaded['input'].splitlines()
# inputdata = open('input.txt', 'r').read().splitlines()

In [5]:
from IPython.display import display

display(f'{inputdata[:10]} ... {len(inputdata)=}')

"['.##.#.#......###..######...##.##.#..#.#.#.#######.###.#####...#...#..#', '#...######.#.#.#.##....#..##.###.#..##.#..#.....##.....#....###.##.##.', '.##.#.##..#.....#...###..#...##.##...........#.#..###.###...####.##..#', '##.#.##.#.####......#####..#.....#..#....###..#..####.##.#.###....####', '.....#...#.######...###..#....##.#.####.###.#..###...#.#.#..###.##...#', '#.#...#.##.....#..#######..##.###.###.####.#.##..##........#.#####..#.', '.#..###..#.....#..##.....#..#.##....#....##..###..#####.#....##.#.#..#', '#..##.##.#.##.#..#.#.#.#..#.#.#..#.#...######.###.#..##.##...##.#.##.#', '.##....#....#..#..#.###.......##.#..###..#....#.####....##.#.##...###.', '.....#.###.#....##.....#...###...##.#.#.#.....###.#..#..##..#..##.#.##'] ... len(inputdata)=70"

In [6]:
from typing import List, Tuple, Set

Coord = Tuple[int, int]


def parse_data(data: List[str]):
    elves: Set[Coord] = set()
    for row, line in enumerate(data):
        for column, character in enumerate(line):
            if character == '#':
                elves.add((row, column))
    return elves

In [20]:
from typing import Dict, Optional

DIRECTIONS: Dict[str, Coord] = {
    'NW': (-1, -1), 'N': (-1, 0), 'NE': (-1, 1),
    'W': (0, -1), 'E': (0, 1),
    'SW': (1, -1), 'S': (1, 0), 'SE': (1, 1)}


def add_move(elf: Coord, direction: Coord) -> Coord:
    return (elf[0] + direction[0], elf[1] + direction[1])


def neighbours(elf: Coord, elves: Set[Coord]) -> Dict[str, Coord]:
    output: Dict[str, Coord] = dict()
    for dir_name, dir_off in DIRECTIONS.items():
        neighbour = add_move(elf, dir_off)
        if neighbour in elves:
            output[dir_name] = neighbour
    return output


# If there is no Elf in the N, NE, or NW adjacent positions,
# the Elf proposes moving north one step.
# If there is no Elf in the S, SE, or SW adjacent positions,
# the Elf proposes moving south one step.
# If there is no Elf in the W, NW, or SW adjacent positions,
# the Elf proposes moving west one step.
# If there is no Elf in the E, NE, or SE adjacent positions,
# the Elf proposes moving east one step.
def propose_move(elf: Coord, elves: Set[Coord]) -> Optional[Coord]:
    ns = neighbours(elf, elves)
    if not any(dir_name in ns.keys() for dir_name in ['N', 'NE', 'NW']):
        return add_move(elf, DIRECTIONS['N'])
    elif not any(dir_name in ns.keys() for dir_name in ['S', 'SE', 'SW']):
        return add_move(elf, DIRECTIONS['S'])
    elif not any(dir_name in ns.keys() for dir_name in ['W', 'NW', 'SW']):
        return add_move(elf, DIRECTIONS['W'])
    elif not any(dir_name in ns.keys() for dir_name in ['E', 'NE', 'SE']):
        return add_move(elf, DIRECTIONS['E'])
    else:
        return None


def agree_moves(elves: Set[Coord]) -> Dict[Coord, Coord]:
    moves_by: Dict[Coord, Coord] = dict()
    collisions: Set[Coord] = set()
    for elf in elves:
        proposed_move = propose_move(elf, elves)
        if proposed_move:
            if (proposed_move not in moves_by.keys() and
                    proposed_move not in collisions):
                moves_by[proposed_move] = elf
            else:
                moves_by.pop(proposed_move)
                collisions.add(proposed_move)
    return {v: k for k, v in moves_by.items()}


def do_moves(elves: Set[Coord], moves: Dict[Coord, Coord]) -> Set[Coord]:
    output = elves.copy()
    for move_from, move_to in moves.items():
        output.remove(move_from)
        output.add(move_to)
    return output


def do_round(elves: Set[Coord]) -> Set[Coord]:
    return do_moves(elves, agree_moves(elves))


def count_spaces(elves: Set[Coord]) -> int:
    min_row = min(e[0] for e in elves)
    max_row = max(e[0] for e in elves)
    min_col = min(e[1] for e in elves)
    max_col = max(e[1] for e in elves)
    return (max_row - min_row + 1)*(max_col - min_col + 1) - len(elves)

In [22]:
elves = parse_data(testdata)
# {propose_move(elf, elves) for elf in elves}
for _ in range(11):
    elves = do_round(elves)
count_spaces(elves)

104

In [8]:
HTML(downloaded['part1_footer'])

## Part Two

In [9]:
# HTML(downloaded['part2'])

In [10]:
# HTML(downloaded['part2_footer'])